In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import os 
os.chdir('/Users/kimmh/Desktop/서데캠/')
os.getcwd()

'/Users/kimmh/Desktop/서데캠'

In [3]:
dt = pd.read_table('./data/TB_T_RSPOP_ADMI.txt',sep='|')
dt.columns = ['STD_YM', 'ADMI_CD', 'ADMI_NM', 'SEXDSTN_CD', 'AGRDE_CD', 'RSPOP_CNT', 'CTY_NM']
dt.head()

,STD_YM,ADMI_CD,ADMI_NM,SEXDSTN_CD,AGRDE_CD,RSPOP_CNT,CTY_NM
0,`201807`,`11230660`,`장안2동`,`1`,`0004`,`667`,`동대문구`
1,`201807`,`11590660`,`대방동`,`1`,`0004`,`599`,`동작구`
2,`201807`,`11170700`,`보광동`,`1`,`0004`,`156`,`용산구`
3,`201807`,`11290525`,`성북동`,`1`,`0004`,`210`,`성북구`
4,`201807`,`11620585`,`낙성대동`,`1`,`0004`,`228`,`관악구`


* STD_YM: 기준년월
* ADMI_CD: 행정동코드 8자리
* ADMI_NM: 행정동명
* SEXDSTN_CD: 성별 코드 (1=남, 2=여) 
* AGRDE_CD: 연령대 코드 
* RSPOP_CNT: 거주인구 수
* CTY_NM: 시군구명

In [4]:
pd.Series(dt['STD_YM'].unique()).sort_values()

14     `201112`
15     `201201`
1      `201202`
16     `201203`
17     `201204`
         ...   
110    `202102`
111    `202104`
112    `202105`
113    `202106`
114    `202107`
Length: 115, dtype: object

**2011년 12월부터 2021년 7월까지 총 (1+12*9+7) = 116 이어야 하는데 왜 115일까?  
2021년 3월 데이터가 없음 시발..** 

In [5]:
why_115 = dt[dt['STD_YM']==202103]
why_115

,STD_YM,ADMI_CD,ADMI_NM,SEXDSTN_CD,AGRDE_CD,RSPOP_CNT,CTY_NM


In [6]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2046404 entries, 0 to 2046403
Data columns (total 7 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   STD_YM      object
 1   ADMI_CD     object
 2   ADMI_NM     object
 3   SEXDSTN_CD  object
 4   AGRDE_CD    object
 5   RSPOP_CNT   object
 6   CTY_NM      object
dtypes: object(7)
memory usage: 109.3+ MB


In [7]:
dt['SEXDSTN_CD'].value_counts()  # 성별 1=남 2=여

`1`    1023202
`2`    1023202
Name: SEXDSTN_CD, dtype: int64

In [8]:
dt['AGRDE_CD'].value_counts()  # 연령을 5살 기준으로 나눈 코드 (총 22가지)

`9599`    97448
`1014`    97448
`6064`    97448
`2529`    97448
`3034`    97448
`6569`    97448
`3539`    97448
`2024`    97448
`7074`    97448
`0509`    97448
`8084`    97448
`1519`    97448
`4549`    97448
`9094`    97448
`5559`    97448
`5054`    97448
`4044`    97448
`8589`    97448
`7579`    97448
`0004`    97444
`9999`    50810
`I00`     46638
Name: AGRDE_CD, dtype: int64

#### 성별 / 연령이 필요할까?  일단 제거  
42가지 경우의 수 제거됨 

In [9]:
dt = dt.drop(['SEXDSTN_CD', 'AGRDE_CD'], axis=1)
dt.head()

,STD_YM,ADMI_CD,ADMI_NM,RSPOP_CNT,CTY_NM
0,`201807`,`11230660`,`장안2동`,`667`,`동대문구`
1,`201807`,`11590660`,`대방동`,`599`,`동작구`
2,`201807`,`11170700`,`보광동`,`156`,`용산구`
3,`201807`,`11290525`,`성북동`,`210`,`성북구`
4,`201807`,`11620585`,`낙성대동`,`228`,`관악구`


#### 양 옆의 따옴표 다 제거

In [10]:
# dt = dt.apply(lambda x: x.str.strip('`'), axis = 1)   >>> 너무 오래 걸려서 혹시나 코드 돌려볼 때 방해될까봐 주석처리 

In [21]:
# dt['RSPOP_CNT'] = pd.to_numeric(dt['RSPOP_CNT'])
# dt.info()

#### 따옴표 제거하여 저장

In [12]:
# dt.to_csv('TB_T_RSPOP_ADMI_processed.txt', sep=',', index=False)  >>> 전처리한 데이터 따로 저장 

---

### 전처리한 데이터로 다시 open

In [26]:
dt = pd.read_table('./data/TB_T_RSPOP_ADMI_processed.txt',sep=',')
dt.head()

,STD_YM,ADMI_CD,ADMI_NM,RSPOP_CNT,CTY_NM
0,201807,11230660,장안2동,667,동대문구
1,201807,11590660,대방동,599,동작구
2,201807,11170700,보광동,156,용산구
3,201807,11290525,성북동,210,성북구
4,201807,11620585,낙성대동,228,관악구


In [27]:
dt.shape

(2046404, 5)

### 일단 2021년 7월 따로 저장해두기

In [34]:
dt_2107 = dt[dt['STD_YM']==202107]
dt_2107['STD_YM'].unique()

array([202107])

In [36]:
dt_new = dt_2107.reset_index(drop=True)
print(dt_new.shape)
dt_new.head()

(17766, 5)


,STD_YM,ADMI_CD,ADMI_NM,RSPOP_CNT,CTY_NM
0,202107,11200540,마장동,265,성동구
1,202107,11290555,삼선동,205,성북구
2,202107,11545510,가산동,177,금천구
3,202107,11560605,문래동,769,영등포구
4,202107,11470560,신월1동,215,양천구


In [37]:
# dt_new.to_csv('TB_T_RSPOP_ADMI_2107.txt', sep = ',', index = False)

---

In [32]:
data = dt.groupby(['STD_YM', 'ADMI_CD', 'ADMI_NM', 'CTY_NM'])
grouped = data.sum('RSPOP_CNT')
grouped = grouped.reset_index()
print(grouped.shape)
grouped.head()

(48724, 5)


,STD_YM,ADMI_CD,ADMI_NM,CTY_NM,RSPOP_CNT
0,201112,11110515,청운효자동,종로구,15260
1,201112,11110530,사직동,종로구,10150
2,201112,11110540,삼청동,종로구,3415
3,201112,11110550,부암동,종로구,11369
4,201112,11110560,평창동,종로구,19677


In [31]:
dt.shape[0] / grouped.shape[0]

41.999917904933916

약 42 = 성별 2가지 * 연령 22가지 (연령 구분 2가지가 없는 경우도 있어서)

In [16]:
grouped['ADMI_NM'].value_counts() 

신사동      230
쌍문3동     115
창1동      115
목3동      115
노량진1동    115
        ... 
신당4동      19
신당6동      19
신당3동      19
신당1동      19
명륜3가동      8
Name: ADMI_NM, Length: 429, dtype: int64

명륜3가동 등 너무 적어서 뭐지 싶었는데  
원본 데이터의 명륜3가동 row 수가 336으로 8 * 42 로 맞아 떨어지는 것으로 보아  
새로 생긴 행정동 이름인 듯

In [18]:
grouped['ADMI_NM'].value_counts().head()

신사동      230
쌍문3동     115
창1동      115
목3동      115
노량진1동    115
Name: ADMI_NM, dtype: int64

In [19]:
grouped['ADMI_NM'].unique() 

array(['청운효자동', '사직동', '삼청동', '부암동', '평창동', '무악동', '교남동', '가회동',
       '종로1.2.3.4가동', '종로5.6가동', '이화동', '혜화동', '명륜3가동', '창신1동', '창신2동',
       '창신3동', '숭인1동', '숭인2동', '소공동', '회현동', '명동', '필동', '장충동', '광희동',
       '을지로동', '신당1동', '신당2동', '신당3동', '신당4동', '신당5동', '신당6동', '황학동',
       '중림동', '후암동', '용산2가동', '남영동', '청파동', '원효로1동', '원효로2동', '효창동',
       '용문동', '한강로동', '이촌1동', '이촌2동', '이태원1동', '이태원2동', '한남동', '서빙고동',
       '보광동', '왕십리2동', '왕십리도선동', '마장동', '사근동', '행당1동', '행당2동', '응봉동',
       '금호1가동', '금호2.3가동', '금호4가동', '옥수동', '성수1가1동', '성수1가2동', '성수2가1동',
       '성수2가3동', '송정동', '용답동', '화양동', '군자동', '중곡1동', '중곡2동', '중곡3동',
       '중곡4동', '능동', '광장동', '자양1동', '자양2동', '자양3동', '자양4동', '구의1동',
       '구의2동', '구의3동', '용신동', '제기동', '전농1동', '전농2동', '답십리1동', '답십리2동',
       '장안1동', '장안2동', '청량리동', '회기동', '휘경1동', '휘경2동', '이문1동', '이문2동',
       '면목2동', '면목4동', '면목5동', '면목본동', '면목7동', '면목3.8동', '상봉1동', '상봉2동',
       '중화1동', '중화2동', '묵1동', '묵2동', '망우본동', '망우3동', '신내1동', '신내2동',
       '성북동', '삼선동'

'종로1.2.3.4가동', '종로5.6가동', '금호2.3가동', '중계2.3동', '상계3.4동' >>> 얘넨 뭐여

In [20]:
grouped[grouped.ADMI_NM.str.contains(r'(종로)')]

/Users/kimmh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,STD_YM,ADMI_CD,ADMI_NM,CTY_NM,RSPOP_CNT
8,201112,11110615,종로1.2.3.4가동,종로구,7923
9,201112,11110630,종로5.6가동,종로구,6156
432,201201,11110615,종로1.2.3.4가동,종로구,7957
433,201201,11110630,종로5.6가동,종로구,6192
856,201202,11110615,종로1.2.3.4가동,종로구,7943
...,...,...,...,...,...
47462,202105,11110630,종로5.6가동,종로구,5243
47885,202106,11110615,종로1.2.3.4가동,종로구,7226
47886,202106,11110630,종로5.6가동,종로구,5150
48309,202107,11110615,종로1.2.3.4가동,종로구,7204


In [38]:
grouped.sort_values(by='RSPOP_CNT', axis=0, ascending=False)

,STD_YM,ADMI_CD,ADMI_NM,CTY_NM,RSPOP_CNT
45100,202010,11380690,진관동,은평구,114350
45153,202010,11500540,화곡1동,강서구,105096
45095,202010,11380625,역촌동,은평구,93542
45330,202010,11740685,길동,강동구,93282
45145,202010,11470640,신정3동,양천구,93266
...,...,...,...,...,...
47027,202102,11740690,둔촌1동,강동구,247
47451,202104,11740690,둔촌1동,강동구,134
48722,202107,11740690,둔촌1동,강동구,134
48299,202106,11740690,둔촌1동,강동구,134


* 둔촌1동 : 학교 3곳이랑 주공아파트 단지 뿐인데 이 주공아파트가 재건축 들어가서 저렇게 낮게 나옴 
* 진관동 등등 : 네이버 지도 검색해보니 면적이 매우 큼 (특히 1,2,3... 이렇게 안 나뉜 곳들)